In [ ]:
# 天氣資訊
# import 相關模組
# 連線到 中央氣象署開放資料平台 並擷取相關 API
import requests
from tqdm import tqdm
from openpyxl import Workbook, load_workbook


authorization_code = "CWA-7F39851F-2584-487A-A19A-7D0CF824673B"
url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-D0047-091"
res = requests.get(url, {"Authorization": authorization_code})

In [ ]:
# raw data
res_json = res.json()
all_data = res_json["records"]["locations"][0]["location"]

titles = [
    "County",
    "longitude",
    "latitude",
    "Start_Time",
    "End_time",
    "PoP12h",
    "T",
    "RH",
    "minCI",
    "WS",
    "MaxAT",
    "Wx",
    "MaxCI",
    "minT",
    "UVI",
    "WeatherDescription",
    "MinAT",
    "MaxT",
    "WD",
    "TD",
]

# 建立工作表
def create_wb(path):
    new_wb = Workbook()
    new_wb.save(path)

saving_path = input("please give a path")
create_wb(saving_path)

# 載入工作表
wb = load_workbook(saving_path)
ws = wb.active
ws.title = "weather-info-data"

# 寫入表頭
ws.append(titles)

for county_info in tqdm(all_data):
    temp_result = [[], [], [], [], [], [], [], [], [], [], [], [], [], []]
    county_name = county_info["locationName"]  # 縣市名
    county_lat = county_info["lat"]  # 緯度
    county_lon = county_info["lon"]  # 經度

    weather_info = county_info["weatherElement"]

    for weather_element in weather_info:
        weather_factor = weather_element["elementName"]

        weather_time = weather_element["time"]
        for i in range(len(weather_time)):

            value = weather_time[i]["elementValue"][0]["value"]
            unit = weather_time[i]["elementValue"][0]["measures"]

            # print(unit, value)
            if "自定義" in unit:
                unit = ""
            result = value + unit
            if value == " ":
                result = "null"
            elif "NA" in result:
                result = result.replace("NA", "")

            ## 紫外線特例例外處理
            if weather_factor == "UVI" and i == 6:
                for j in range(7, 14):
                    temp_result[j].append("null")

            temp_result[i].append(result)

    for i in range(len(temp_result)):
        start_time = weather_time[i]["startTime"]
        end_time = weather_time[i]["endTime"]
        row = [county_name, county_lon, county_lat, start_time, end_time] + temp_result[
            i
        ]
        ws.append(row)

# 關閉工作表
wb.save(saving_path)